# 3I025 &mdash; Projet : Cooperative Path-Finding

Thierry UNG 

Dans ce projet on met en œuvre des algorithmes pour la recherche coopérative de chemins sur une carte. L’idée est de trouver une façon de bouger des personnages vers des objectifs sans qu’il y ait des collisions entre eux. La base de ces algorithmes est l’algorithme A* et chacun d’entre eux implémente des techniques supplémentaires différentes pour empêcher des collisions.

Dans le modèle utilisé chaque joueur a connaissance de la carte et aussi de la position de tous les autres joueurs. Ils connaîssent aussi le chemin choisi par les autres.  

On a codé 3 algorithmes de recherche coopérative : 

- Une stratégie opportuniste qui recalcule des parties de chemins quand elle détecte une collision dans quelques pas, dite Slicing. Chaque chemin est donc calculé dans un premier temps sans se soucier des chemins et positions des autres;

- Une stratégie coopérative de base qui essaie d’exécuter la plus grande quantité possible de chemins en parallèle, en considérant une collision comme un croisement de chemins sans prendre en compte le temps;

- Une stratégie coopérative plus avancée qui calcule les chemins des joueurs en considérant à quel pas de temps ils passeront sur une case donnée. On a fait 2 versions de cette stratégie, une dans laquelle le chemin de chaque joueur est calculé jusqu’à son objectif et une deuxième où le chemin est recalculée à chaque `d` pas de temps. 

In [1]:
import main

pygame 2.0.1 (SDL 2.0.14, Python 3.8.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Stratégie de slicing

Dans cette stratégie, on commence par calculer un chemin jusqu’à l'objectif pour chaque joueur sans prendre en considération le chemin des autres. Cela est fait par une implémentation de l’algorithme A\* en utilisant la distance de Manhattan comme heuristique. Il s’agit d’une heuristique admissible pour ce problème donc on sait que A\* retourne bien une solution optimale. Comme elle est aussi consistante on sait que pour chaque point qui est sorti du tas, sa distance est bien optimale et il ne rentrera plus jamais dans le tas, alors on peut faire une sortie anticipée de la méthode dès que l’on retire la position de la cible du tas.  

Une fois les chemins calculés, on les exécute de façon parallèle. Tous les `n` pas de temps, l’algorithme regarde, pour chaque chemin, `m` pas de temps dans le futur pour voir s’il y aura une collision. Si c’est le cas, il recalcule le morceau de chemin entre les positions actuelle et dans `m` pas et remplace dans le chemin les `m` pas suivants par ce nouveau morceau. S’il n’arrive pas à trouver un tel chemin ou si le chemin trouvé est trop long (plus grand ou égal à `max_slice*m`), on recalcule toute la trajectoire jusqu’à la cible. S'il ne trouve toujours pas un chemin possible, le joueur reste immobile pour les `n` prochains pas. On fait en sorte qu’un joueur qui doit rester immobile est considéré comme un obstacle par les autres joueurs, ce qui les empêche d'entrer en collision.

Les valeurs de `m`, `n` et `max_slice` sont passées en argument au constructeur de la classe `Slicing` qui implémente l’algorithme. On doit avoir `m > n` si on veut être certain que tous les morceaux des chemins seront analysés pour éviter des collisions. 

Cette stratégie donne de très bons résultats dans plusieurs cas mais elle ne permet pas d’éviter certaines situations de blocage : Par exemple, si deux joueurs veulent traverser un tunnel plus long que `m` dans des sens opposés, ils finissent par rester immobiles.

Les cellules suivantes donnent les exécutions de chacune des cartes avec cette stratégie :

In [2]:
p = main.Projet(boardNumber = 0, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(13, 13), (12, 2)], verbose = False)

In [3]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(19, 18), (19, 11), (19, 5),(0, 17), (0, 11), (0, 6)], verbose = False)

In [4]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [5]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [6]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [7]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(18, 18), (10, 11), (8, 10), (3, 0)], verbose = False)

In [8]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5)], verbose = False)

In [9]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [10]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Le prochain exemple montre une situation de blocage :

In [11]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], verbose = False)

Le blocage est levé si on augmente la valeur de `m` :

In [12]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], m = 5, verbose = False)

In [13]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], verbose = False)

Le prochain exemple montre une situation de blocage qui ne peut pas être résolue avec une augmentation de m :

In [14]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(9, 9), (4, 7)], verbose = False)

Cet exemple produit une blocage car les cibles sont sur le chemin unique possible pour un joueur :

In [15]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(8, 8), (8, 11)], verbose = False)

Ici un joueur est obligé de recalculer son chemin plusieurs fois ce qui augmente la quantité de pas nécessaires :

In [16]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [17]:
p = main.Projet(boardNumber = 14, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ici aussi on a une blocage puisque les joueurs se bloquent les uns les autres :

In [18]:
p = main.Projet(boardNumber = 15, fps = 5, iterations = 70)
p.mainSlicing(goalStates = [(5, 2), (0, 0)], verbose = False)

Ici on a une carte impossible à résoudre pour tous nos algos :

In [19]:
#p = main.Projet(boardNumber = 16, fps = 5, iterations = 70)
#p.mainSlicing(goalStates = [(5, 2), (1, 2)], verbose = False)

## Stratégie coopérative de base

Cette deuxième stratégie essaie de prendre en considération dans l’exécution d’un chemin les positions des autres joueurs. Cela est fait en prenant les chemins des autres joueurs comme s’ils étaient des obstacles et en ne jouant en parallèle que les joueurs qui n’ont pas de collisions entre eux. Cela produit des résultats positifs mais généralement très longs par rapport à la première stratégie. Pour cette stratégie, `self.iterations` est le nombre maximal d'itérations **par groupe**. Le nombre maximal d'itérations peut être au pire cas égal à nbPlayers * self.iterations.

Plus précisément, on commence par calculer un chemin pour chaque joueur avec l’algorithme A\* en considérant les positions initiales et les cibles des autres joueurs comme des obstacles pour éviter des collisions. Ces chemins ne seront pas modifiés et l’algorithme se contante de trouver un ordre d’exécution sans collisions. Pour ce faire, il décompose l’ensemble des chemins en groupes qui peuvent être exécutés en parallèle. D’abord, on calcule la matrice de collisions, une matrice carrée de taille `n` où `n` est la quantité de joueurs, telle que la case`[i, j]` contient 1 si les chemins `i` et` j` sont en collision, 0 sinon et -1 si `i = j`. On utilise cette matrice pour déterminer quel est le chemin avec le moins de collisions et ce chemin est pris comme premier chemin du premier groupe. Ce groupe commence en ne contenant que ce chemin et ensuite d'autres chemins sont ajoutés successivement s'ils ne croisent pas les chemins déjà présents. On répète cette construction jusqu’à ce que tous les chemins soient dans un groupe et les groupes sont exécutés séquentiellement.

Les collisions sont détectées de façon naïve : deux chemins sont en collision s'ils ont au moins une position en commun, sans prendre en compte les instants où les joueurs passent par cette position.

In [20]:
p = main.Projet(boardNumber = 0, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(13, 13), (12, 2)], verbose = False)

In [21]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(19, 18), (19, 11), (19, 5),(0, 17), (0, 11), (0, 6)], verbose = False)

In [22]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [23]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [24]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [25]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(18, 18), (10, 11), (8, 10), (3, 0)], verbose = False)

In [26]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5)], verbose = False)

Dans les trois prochains cas touts les chemins se croisent entre eux et alors ils sont exécutés de façon séquentielle :

In [27]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [28]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Ce cas était un blocage pour le `Slicing` mais fonctionne avec cette stratégie (on se contente d'exécuter les chemins à la suite et non plus en parallèle) :

In [29]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(0, 9), (0, 1)], verbose = False)

In [30]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(0, 9), (0, 1)], verbose = False)

Ici on a une carte impossible à résoudre :

In [31]:
#p = main.Projet(boardNumber = 11, fps = 5, iterations = 70)
#p.mainCoopBase(goalStates = [(9, 9), (4, 7)], verbose = False)

Ici aussi :

In [32]:
#p = main.Projet(boardNumber = 12, fps = 5, iterations = 70)
#p.mainCoopBase(goalStates = [(8, 8), (8, 11)], verbose = False)

In [33]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [34]:
p = main.Projet(boardNumber = 14, fps = 5, iterations = 70)
p.mainCoopBase(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ce cas reste impossible aussi :

In [35]:
#p = main.Projet(boardNumber = 15, fps = 5, iterations = 70)
#p.mainCoopBase(goalStates = [(5, 2), (0, 0)], verbose = False)

## Stratégie coopérative avancée

La stratégie coopérative de base permet d’éviter des collisions mais comme elle ne prend pas en considération le temps, les solutions données sont souvent très longues. Pour l’améliorer, on prend en considération le temps en suivant les idées données dans [cet article](https://www.davidsilver.uk/wp-content/uploads/2020/03/coop-path-AIWisdom.pdf). L’idée principale est de considérer une dimension supplémentaire pour le temps, ce que l’on fait en implémentant le *Tableau de réservations* de l’article. Nos obstacles sont ainsi décomposés en deux : les obstacles fixes qui ne possèdent pas une coordonnée de temps et les obstacles mobiles (les autres joueurs).

Chaque joueur calcule son plus court chemin à sa cible en utilisant l’algorithme A\* adapté pour prendre en compte la dimension temporelle. Le premier joueur n’a aucun obstacle mobile, le deuxième prend en compte le chemin du premier et ainsi de suite. Alors, on a une priorité naturelle du premier joueur.  

Comme mentionné dans l’article, pour éviter des calculs trop longs, on utilise comme heuristique la distance réelle, obtenue par une application de l’algorithme A\* en espace avec uniquement les obstacles fixes et dans le sens inversé, en partant de la cible et cherchant le chemin le plus court vers la position initiale du joueur.

In [36]:
p = main.Projet(boardNumber = 0, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(13, 13), (12, 2)], verbose = False)

In [37]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(19, 18), (19, 11), (19, 5),(0, 17), (0, 11), (0, 6)], verbose = False)

In [38]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [39]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [40]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [41]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(18, 18), (10, 11), (8, 10), (3, 0)], verbose = False)

In [42]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5)], verbose = False)

In [43]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [44]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Ces deux prochains cas ont un comportement très intéressant :

In [45]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(0, 9), (0, 1)], verbose = False)

In [46]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(0, 9), (0, 1)], verbose = False)

Ce cas était insoluble avant mais cette stratégie arrive à le résoudre (mais seulement car le joueur rouge à la priorité) :

In [47]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(9, 9), (4, 7)], verbose = False)

L’exemple suivant reste insoluble mais ne produit plus d’erreur :

In [48]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(8, 8), (8, 11)], verbose = False)

In [49]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [50]:
p = main.Projet(boardNumber = 14, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ce cas peut désormais être résolu :

In [51]:
p = main.Projet(boardNumber = 15, fps = 5, iterations = 70)
p.mainTempA(goalStates = [(5, 2), (0, 0)], verbose = False)

## Stratégie coopérative avancée avec profondeur de recherche fixée

On modifie la stratégie de coopération temporelle pour arrêter la recherche au bout d’un nombre fixe `d` de pas. Cela a quelques avantages : 

- La priorité n’est plus fixée par l’ordre des joueurs mais change au cours du temps;

- On n’a pas besoin de calculer toutes les trajectoires jusqu’à leurs cibles d’un seul coup, ce qui réduit l’espace mémoire utilisé;

- Elle permet de débloquer quelques autres situations où l’objectif d’un des joueurs est dans le chemin d’un autre.

Pour implémenter cette méthode, on crée un tableau supplémentaire pour stocker les prochaines étapes de la trajectoire des joueurs. À chaque pas de temps, si la case correspondant à un joueur est vide, on calcule les `d` prochains pas de sa trajectoire à l’aide d’un algorithme A\* modifié pour s’arrêter dès que la première case contenant un temps `t = t0 + d` est trouvée, où `t0` est l’instant où cet algorithme est appelé. Pour éviter que les joueurs fassent leurs recalculs au même moment, on utilise des valeurs différentes de `d` pour chaque joueur à la première fois qu’on calcule des chemins. 

On fait aussi un traitement du cas où un joueur n’arrive pas à trouver un chemin valable lorsqu’il recalcule sa trajectoire : on efface la trajectoire future de tous les autres joueurs et on les recalcule en commençant par le joueur qui a déclenché le problème pour lui donner la priorité maximale. 

In [52]:
p = main.Projet(boardNumber = 0, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(13, 13), (12, 2)], verbose = False)

In [53]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(19, 18), (19, 11), (19, 5),(0, 17), (0, 11), (0, 6)], d = 20, verbose = False)

In [54]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [55]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [56]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [57]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(18, 18), (10, 11), (8, 10), (3, 0)], verbose = False)

In [58]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(6, 14), (6, 13), (6, 12), (6, 11), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (6, 5)], verbose = False)

In [59]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [60]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Les 3 prochains cas ont un comportement non optimal en comparaison à la stratégie précédente :

In [61]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], verbose = False)

In [62]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], verbose = False)

In [63]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(9, 9), (4, 7)], verbose = False)

Ils peuvent être améliorés en changeant le paramètre `d` :

In [64]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], d = 11, verbose = False)

In [65]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], d = 11, verbose = False)

In [66]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(9, 9), (4, 7)], d = 8, verbose = False)

Ce cas est finalement résolu grâce à la priorité tournante : 

In [67]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(8, 8), (8, 11)], verbose = False)

Cette prochaine carte était possible avant mais ne l’est plus plus avec le paramètre `d` = 7. Cela démontre que l’algorithme n’est pas toujours la meilleur choix entre les algorithmes codés.

In [68]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

Néanmoins, en changeant d, on arrive à la résoudre avec cet algorithme :

In [69]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], d = 11, verbose = False)

In [70]:
p = main.Projet(boardNumber = 14, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

In [71]:
p = main.Projet(boardNumber = 15, fps = 5, iterations = 70)
p.mainTempA_D(goalStates = [(5, 2), (0, 0)], verbose = False)

## Comparaison des stratégies

Le tableau suivant donne le nombre de pas nécessaires pour que les joueurs atteignent leurs cibles :

Carte | Slicing | Coop Base | Coop Avancé | Coop Avancé D
------|---------|-----------|-------------|--------------
0 | 20 | 37 | 20 | 20
1 | 55 | 176 | 34 | 34 (d = 20)
2 | 22 | 22 | 22 | 22
3 | 16 | 25 | 16 | 16
4 | 23 | 28 | 14 | 14
5 | 29 | 75 | 29 | 29
6 | 58 | 265 | 46 | 45
7 | 50 | 216 | 41 | 41
8 | 33 | 69 | 27 | 26
9 | 23 (m = 5) | 28 | 23 | 23 (d = 11)
10 | 29 | 28 | 23 | 23 (d = 11)
11 | block | noPath | 20 | 20 (d = 8)
12 | block | noPath | block | 26
13 | 33 | 32 | 17 | 17 (d = 11)
14 | 10 | 30 | 10 | 10
15 | block | noPath | 12 | 12
16 | noPath | noPath | noPath | noPath
Les paramètres utilisés sont ceux par défaut sauf lorsque cela est indiqué entre parenthèses. 

On voit que la stratégie coopérative de base a toujours les pires résultats parce que plusieurs joueurs sont trop longtemps immobiles. Les deux stratégies avancées améliorent beaucoup par rapport aux autres et permettent de débloquer certains cas. 

On observe que toutes les méthodes sont très rapides. 